In [1]:
import sys
sys.path.insert(0,'..')
import sib
import pandas as pd
import numpy as np

## Order of parameters

### Contacts:

in order: (i, j, t, lambda)

### Observations:
(i,s,t) which is in order:

- node
- state
- time

In [2]:
#name_folder_data = "ProximityModel/"
name_folder_data = "dummy/"
#root_folder_data = "../sibilla/notebooks/data/"
root_folder_data = "./data/"
folder_data = root_folder_data + name_folder_data

In [3]:
file_contacts_name = folder_data +  "transmissions.csv"
contacts_df = pd.read_csv(file_contacts_name, dtype={
                                         "i":np.int,
                                        "j":np.int,
                                        "lamb":np.float,
                                        "t":np.int
                                                })

In [4]:
contacts_df

,i,j,lambda,t
0,2,3,0.02,0
1,2,3,0.02,0
2,1,2,0.02,0
3,2,1,0.02,1
4,1,2,0.02,1
5,0,1,0.02,3


In [5]:
# set the correct order!
contacts_df = contacts_df[["i", "j","t", "lambda"]] 

#removing symmetric contacts
d = {'i':'j','j':'i'}
contacts_df = pd.concat([contacts_df, contacts_df.rename(columns=d)]).drop_duplicates()
contacts_df = contacts_df.sort_values("t")
contacts_df = contacts_df[contacts_df["i"] < contacts_df["j"]]

# time must start from zero
assert(contacts_df["t"].iloc[0] == 0)

contacts = list(contacts_df.to_records(index=False))

tot_contacts = len(contacts);
print(f"number of unique contacts: {tot_contacts}")

number of unique contacts: 4


In [6]:
file_obs_name = folder_data +  "observations.csv"
obs_df = pd.read_csv(file_obs_name,
                    dtype={"s":np.int,
                           "i":np.int,
                           "t_test":np.int})
obs = obs_df[["i","s","t_test"]]
observations = list(obs.to_records(index=False))
tot_obs = len(observations)
print(f"number of observartions {tot_obs}")

number of observartions 2


In [7]:
observations

[(1, 1, 1), (2, 1, 2)]

In [10]:
params = sib.Params(prob_r=sib.Gamma(mu=0.1), pseed=0.01)
params

Params(prob_i=Uniform(1),prob_r=Gamma(1,0.1),pseed=0.01,psus=0.5)

In [19]:
f = sib.FactorGraph(contacts=contacts, observations=observations, params=params)

In [16]:
def callback(t,err,f):
    print(t,err,end="\r")

In [20]:
sib.iterate(f, maxit=100,tol=1e-5,callback=callback)

In [22]:
sib.marginal(f.nodes[0])

[(0.6929678399733201, 0.3070321600266799, 0.0),
 (0.360300828429807, 0.639699171570193, 0.0),
 (0.0, 1.0, 0.0)]